In [1]:
import pause 
import networkx as nx 
import math
import matplotlib.pyplot as plt
import bellmanford as bf 
import numpy as np
import copy 
from datetime import datetime
from networkx import NetworkXUnbounded
from collections import defaultdict
import matplotlib.pyplot as plt 
from binance.client import Client
import datetime 
client = Client('trJtay6fLLhfd2P67b5OY7vzN6EFR4kcJbk0utuCKO9Io5VtTbbvVQ9kIjPmJrv7', 'tQwbcGQTQbZvzeZPXutp6TAKImejSwSye73meNSTXBE80DEuxEoLeIokwEvAwpXt')

In [2]:
def bellman_ford_negative_cycles(g, s):
    """
    Bellman Ford, modified so that it returns cycles.
    Runtime is O(VE).
    :param g: graph
    :type g: networkx weighted DiGraph
    :param s: source vertex
    :type s: str
    :return: all negative-weight cycles reachable from a source vertex
    :rtype: str list (empty if no neg-weight cyc)
    """
    n = len(g.nodes())
    d = defaultdict(lambda: math.inf)  # distances dict
    p = defaultdict(lambda: -1)  # predecessor dict
    d[s] = 0

    for _ in range(n - 1):
        for u, v in g.edges():
            # Bellman-Ford relaxation
            weight = g[u][v]["weight"]
            if d[u] + weight < d[v]:
                d[v] = d[u] + weight
                p[v] = u  # update pred

    # Find cycles if they exist
    all_cycles = []
    seen = defaultdict(lambda: False)

    for u, v in g.edges():
        weight = g[u][v]["weight"]
        # If we can relax further, there must be a neg-weight cycle
        if seen[v]:
            continue

        if d[u] + weight < d[v]:
            cycle = []
            x = v
            while True:
                # Walk back along predecessors until a cycle is found
                seen[x] = True
                cycle.append(x)
                x = p[x]
                if x == v or x in cycle:
                    break
            # Slice to get the cyclic portion
            idx = cycle.index(x)
            cycle.append(x)
            all_cycles.append(tuple(cycle[idx:][::-1]))
    return all_cycles


def all_negative_cycles(g):
    """
    Get all negative-weight cycles by calling Bellman-Ford on
    each vertex. O(V^2 E)
    :param g: graph
    :type g: networkx weighted DiGraph
    :return: list of negative-weight cycles
    :rtype: list of str list
    """
    all_paths = []
    for v in g.nodes():
        all_paths.append(bellman_ford_negative_cycles(g, v))
    #flatten = lambda l: [item for sublist in l for item in sublist]
    #return [list(i) for i in set(tuple(j) for j in flatten(all_paths))]
    return (all_paths)


def calculate_arb(cycle, g, verbose=True):
    """
    For a given negative-weight cycle on the log graph, calculate and
    print the arbitrage
    :param cycle: the negative-weight cycle
    :type cycle: list
    :param g: graph
    :type g: networkx weighted DiGraph
    :param verbose: whether to print path and arb
    :type verbose: bool
    :return: fractional value of the arbitrage
    :rtype: float
    """
    total = 0
    for (p1, p2) in zip(cycle, cycle[1:]):
        total += g[p1][p2]["weight"]
    arb = np.exp(-total) - 1
    if verbose:
        print("Path:", cycle)
        print(f"{arb*100:.2g}%\n")
    return arb

In [5]:
def find_arbitrage(find_all=False, sources=None):
    """
    Looks for arbitrage opportunities within a snapshot, i.e negative-weight cycles
    that include the currencies given in the sources list
    :param filename: filename of snapshot, defaults to "snapshot.csv"
    :type filename: str, optional
    :param find_all: whether to find all paths, defaults to False.
                     If false, sources must be provided.
    :type find_all: bool, optional
    :param sources: list of starting nodes – should choose the 'most connected' pairs, 
                    defaults to None.
    :type sources: str list, optional
    :return: list of negative-weight cycles, or None if none exist 
    :rtype: str list
    """
    # Read df and convert to negative logs so we can use Bellman Ford
    # Negative weight cycles thus correspond to arbitrage opps
    # Transpose log_df so that graph has same API as the dataframe
    #df = pd.read_csv(filename, header=0, index_col=0)
    #g = nx.DiGraph(-np.log(df).fillna(0).T)
    g = nx.DiGraph()

    '''number of nodes and edges seems to be correct but still getting differences in list lengths'''

    live = client.get_exchange_info()
    symbols = live['symbols']

    ticks = client.get_orderbook_tickers()

    ticklist = []
    for tick in ticks:
        if tick['bidPrice'] != '0.00000000' and tick['bidQty'] != '0.00000000' and tick['askPrice'] != '0.00000000' and tick['askQty'] != '0.00000000': 
            ticklist.append(tick)

    approvedsymbols = []
    for tick in ticklist: 
        approvedsymbols.append(tick['symbol'])

    #print(live['symbols'][0]['symbol'])   
    #print(len(approvedsymbols))


    pairs = []
    for i in range(len(ticklist)):
        if symbols[i]['symbol'] in approvedsymbols:
            temp = []
            temp.append(symbols[i]['baseAsset'])
            temp.append(symbols[i]['quoteAsset'])
            pairs.append(temp)

    """forward entails traversing to the right through a pair (ETH --> USD) for (ETH,USD)"""
    forwardweights = []
    for i in range(len(ticklist)):
        forwardweights.append(float(ticklist[i]['bidPrice']))

    """backward entails traversing to the left through a pair (USD --> ETH) for (ETH,USD)"""
    backwardweights = []
    for i in range(len(ticklist)):
        backwardweights.append(1/float(ticklist[i]['askPrice']))

    """generating network from data """
    forwardedges = copy.deepcopy(pairs)
    backwardedges = copy.deepcopy(forwardedges)

    for edge in backwardedges:
        temp = edge[0] 
        edge.pop(0)
        edge.append(temp) 

    nodes = []
    for pair in forwardedges: 
        for node in pair: 
            if node not in nodes: 
                nodes.append(node)

    """adding weights -- have to do this after creating nodes"""  

    forward_weighted_edges = copy.deepcopy(forwardedges)
    backward_weighted_edges = copy.deepcopy(backwardedges)


    for i in range(len(forward_weighted_edges)):
        forward_weighted_edges[i].append((-1 * math.log(forwardweights[i])))

    for i in range(len(backward_weighted_edges)): 
        backward_weighted_edges[i].append((-1 * math.log(backwardweights[i])))

    g.add_nodes_from(nodes)
    g.add_weighted_edges_from(forward_weighted_edges)
    g.add_weighted_edges_from(backward_weighted_edges)
    if nx.negative_edge_cycle(g):
        print("ARBITRAGE FOUND\n" + "=" * 15 + "\n")

        if find_all:
            unique_cycles = all_negative_cycles(g)
            print(unique_cycles)
            #for cycle in unique_cycles: 
                #print(len(cycle))
        else:
            all_paths = []
            for s in sources:
                all_paths.append(bellman_ford_negative_cycles(g, s))
            flatten = lambda l: [item for sublist in l for item in sublist]
            unique_cycles = [list(i) for i in set(tuple(j) for j in flatten(all_paths))]

        for p in unique_cycles:
            calculate_arb(p, g)
        return unique_cycles

    else:
        print("No arbitrage opportunities")
        return None

In [6]:
now = datetime.datetime.now()
find_arbitrage(find_all = True)
later = datetime.datetime.now()
print(later - now)

ARBITRAGE FOUND

[[('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('XRP', 'EUR', 'BNB', 'VTHO', 'USDT', 'XRP'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('XRP', 'EUR', 'BNB', 'VTHO', 'USDT', 'XRP'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT'), ('EUR', 'BNB', 'VTHO', 'USDT', 'XRP', 'EUR'), ('XRP',

KeyError: ('USDT', 'XRP', 'EUR', 'BNB', 'VTHO', 'USDT')

In [ ]:
now = datetime.datetime.now()
find_arbitrage(find_all = True)
later = datetime.datetime.now()
print(later - now)

In [17]:
g = nx.DiGraph()

'''number of nodes and edges seems to be correct but still getting differences in list lengths'''

live = client.get_exchange_info()
symbols = live['symbols']

ticks = client.get_orderbook_tickers()

ticklist = []
for tick in ticks:
    if tick['bidPrice'] != '0.00000000' and tick['bidQty'] != '0.00000000' and tick['askPrice'] != '0.00000000' and tick['askQty'] != '0.00000000': 
        ticklist.append(tick)

approvedsymbols = []
for tick in ticklist: 
    approvedsymbols.append(tick['symbol'])

#print(live['symbols'][0]['symbol'])   
#print(len(approvedsymbols))


pairs = []
for i in range(len(ticklist)):
    if symbols[i]['symbol'] in approvedsymbols:
        temp = []
        temp.append(symbols[i]['baseAsset'])
        temp.append(symbols[i]['quoteAsset'])
        pairs.append(temp)

"""forward entails traversing to the right through a pair (ETH --> USD) for (ETH,USD)"""
forwardweights = []
for i in range(len(ticklist)):
    forwardweights.append(float(ticklist[i]['bidPrice']))

"""backward entails traversing to the left through a pair (USD --> ETH) for (ETH,USD)"""
backwardweights = []
for i in range(len(ticklist)):
    backwardweights.append(1/float(ticklist[i]['askPrice']))

"""generating network from data """
forwardedges = copy.deepcopy(pairs)
backwardedges = copy.deepcopy(forwardedges)

for edge in backwardedges:
    temp = edge[0] 
    edge.pop(0)
    edge.append(temp) 

nodes = []
for pair in forwardedges: 
    for node in pair: 
        if node not in nodes: 
            nodes.append(node)

"""adding weights -- have to do this after creating nodes"""  

forward_weighted_edges = copy.deepcopy(forwardedges)
backward_weighted_edges = copy.deepcopy(backwardedges)


for i in range(len(forward_weighted_edges)):
    forward_weighted_edges[i].append((-1 * math.log(forwardweights[i])))

for i in range(len(backward_weighted_edges)): 
    backward_weighted_edges[i].append((-1 * math.log(backwardweights[i])))

g.add_nodes_from(nodes)
g.add_weighted_edges_from(forward_weighted_edges)
g.add_weighted_edges_from(backward_weighted_edges)

print((g.nodes))

['ETH', 'BTC', 'LTC', 'BNB', 'NEO', 'QTUM', 'EOS', 'SNT', 'BNT', 'GAS', 'USDT', 'WTC', 'LRC', 'YOYO', 'OMG', 'ZRX', 'SNGLS', 'BQX', 'KNC', 'FUN', 'SNM', 'IOTA', 'LINK', 'XVG', 'MDA', 'MTL', 'ETC', 'MTH', 'DNT', 'ZEC', 'AST', 'DASH', 'OAX', 'BTG', 'EVX', 'REQ', 'VIB', 'TRX', 'POWR', 'ARK', 'XRP', 'ENJ', 'STORJ', 'KMD', 'RCN', 'NULS', 'RDN', 'XMR', 'DLT', 'AMB', 'BAT', 'BCPT', 'GVT', 'CDT', 'GXS', 'QSP', 'BTS', 'XZC', 'LSK', 'MANA', 'BCD', 'ADX', 'ADA', 'PPT', 'CMT', 'XLM', 'CND', 'WABI', 'TNB', 'WAVES', 'GTO', 'ICX', 'OST', 'ELF', 'AION', 'NEBL', 'BRD', 'NAV', 'APPC', 'VIBE', 'RLC', 'PIVX', 'IOST', 'STEEM', 'NANO', 'VIA', 'BLZ', 'AE', 'NCASH', 'POA', 'ZIL', 'ONT', 'XEM', 'WAN', 'WPR', 'QLC', 'SYS', 'GRS', 'GNT', 'LOOM', 'REP', 'TUSD', 'ZEN', 'SKY', 'CVC', 'THETA', 'IOTX', 'QKC', 'AGI', 'NXS', 'DATA', 'SC', 'NPXS', 'KEY', 'NAS', 'MFT', 'DENT', 'ARDR', 'HOT', 'VET', 'DOCK', 'POLY', 'HC', 'GO', 'PAX', 'RVN', 'DCR', 'MITH', 'REN', 'USDC', 'BTT', 'ONG', 'FET', 'CELR', 'MATIC', 'ATOM', 'PHB',